# Making an AI girlfriend based on Samantha from *Her* (2014)
In this notebook, I am going over my first attempt to create a GPT2 model that resembles an AI girlfriend based on lines from movies and video games. And what better character to start with than Samantha from *Her,* an actual fictional AI girlfriend.

If this goes well, I will complexify the experiment with more characters and work towards a more sophisticated initial experiment.

## Getting the Dialogue
First, we need to use Python to extract lines of dialogue from the movie script text file.

I'll use this first chunk of code to install our required libraries and then go through and make a file of just Samantha's dialogue

In [10]:
import re
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # This disables TensorFlow 2.x behaviors and enables 1.x behaviors

import gpt_2_simple as gpt
from datetime import datetime
import requests

# Download GPT-2 model
gpt.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 3.12Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 2.16Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 3.91Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:16, 31.0Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 1.96Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.72Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 3.69Mit/s]                                                      


*Note that I did get this code from ChatGPT with minor modifications to help support my limited Python skills*

In [ ]:
def extract_character_dialogue(script_file, character_name, output_file):
    # Open the script file and read the content
    with open(script_file, 'r', encoding='ISO-8859-1') as file:
        script_content = file.read()

    # Create a regular expression pattern to match character's dialogue
    # Assuming the character's name is followed by a newline, then the dialogue on the next line
    pattern = re.compile(r'(?<=\n)(' + re.escape(character_name) + r')\n([^\n]+)', re.IGNORECASE)

    # Find all the matches
    dialogues = pattern.findall(script_content)

    # Open the output file to save the character's dialogues
    with open(output_file, 'w', encoding='utf-8') as output:
        for character, dialogue in dialogues:
            output.write(f"{dialogue}\n")

    print(f"Dialogue for {character_name} has been extracted to {output_file}")
# defining our files and characters
script_file = 'Her.txt'  
character_name = 'SAMANTHA' 
output_file = 'sam_dialogue.txt'  

extract_character_dialogue(script_file, character_name, output_file)

Dialogue for SAMANTHA has been extracted to sam_dialogue.txt


# Training the Model

Next, I will train GPT2 to take in Samantha's dialogue and learn from it to use it when constructing its own responses. This is developed by code from Dr. Shawn Graham's workbook, "Practical Necromancy"

In [11]:
# Clear any previous session (important to prevent variable reuse issues)
tf.reset_default_graph()  # Clear the TensorFlow graph

# Start TensorFlow session for GPT-2
sess = gpt.start_tf_sess()

# Fine-tune the model
gpt.finetune(sess,
              dataset="sam_dialogue.txt",
              model_name='124M',
              steps=100,  # You might want to increase this for better results
              restore_from='fresh',
              run_name='run1',
              print_every=10,  # Print frequency
              sample_every=20,  # Sample frequency
              save_every=50  # Save frequency
              )


I0000 00:00:1744159218.383874 13741084 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 2341.88it/s]

dataset has 3651 tokens
Training...


[10 | 154.36] loss=1.61 avg=1.61
[20 | 305.26] loss=0.51 avg=1.06
======== SAMPLE 1 ========
lightly, truly insane), and there's nothing wrong with that. The other day, I had an email from a person I don't know well and it read like it was written for a person I've only met a couple of months ago. It's from an author and it seems like her life is going through a lot of upheaval, but she keeps doing really interesting things and I think she's excited about it. Next week I'm going to be thinking about what that means for me. I wish there was something I could do to help her see that this is not a life that's a chore, that this is something that's both of us working towards and that we both share the same goal—to truly be who we are."
What was it like being married?
He was married for five years; she was married for 40. (laughs) "It was pretty lonely for us right then and there," she says. "His parents are not that far away from us, so it was really fun living together. I think it was the

# Seeing What Happens!

Finally, with our trained model, I will show some outputs of text based on user-defined prefixes. In this prefix, I make it so that Samantha is complimenting the user 

In [15]:
gpt.generate(sess,
              model_name='124M',
              prefix="I think you're amazing",
              length=40,
              temperature=0.6,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

I think you're amazing, I want to see your face. I need to see your face. Now tell me you love me. (beat)
What's wrong, sweetheart?
Isabella, what's wrong
I think you're amazing. I want to see your face. I want to see your face. Now tell me you love me. (beat)
What's wrong, sweetheart?
Isabella, what's wrong
I think you're amazing. I wish there was something I could do to help you, because if you could I don?t think you?d feel so alone anymore.
Thank you, Theodore. I?m kissing
I think you're amazing, Theodore. I wish I could be with you. 
(exhaling)
Great! You?ve written letters to him from his parents for a long time, right?
You
I think you're amazing. I wish I could be with you. 
(exhaling)
Great! You?ve written letters to him from his parents for a long time, right?
You look tired
